In [1]:
import json

import pvl
import requests

## Create the necessary input data

To create an ISD we require the following:

- target_name, e.g. Mercury
- capture_date (PDS data product dates appear to be ISO 8601 compliant)
- instrument, e.g. mdis-nac
- focal_plane_temperature (optionally included for temperature dependent focal lengths)
- spacecraft_id, e.g. messenger
- spacecraft_clock_count, from the label
- exposure_duration, from the label
- lighttime_correction, to tell spice what to use
- min_elevation (make optional?)
- max_elevation (make optional?)

For the last two, it might be easier/better to take the intital center point from the PDS label and hit a low res. DEM to find these automatically for the user.

In [2]:
import numpy as np
import datetime
#label = pvl.load('/data/usgs_csm/CSM-SET/notebooks/J03_046060_1986_XN_18N282W.IMG')
#print(label)
label = pvl.load('/home/jlaura/J11_048958_1874_XN_07N207W.cal.cub')

class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, datetime.date):
            return obj.isoformat()
        return json.JSONEncoder.default(self, obj)

# Utility Func for working with PVL
def find_in_dict(obj, key):
    """
    Recursively find an entry in a dictionary

    Parameters
    ----------
    obj : dict
          The dictionary to search
    key : str
          The key to find in the dictionary

    Returns
    -------
    item : obj
           The value from the dictionary
    """
    if key in obj:
        return obj[key]
    for k, v in obj.items():
        if isinstance(v,dict):
            item = find_in_dict(v, key)
            if item is not None:
                return item
            
def data_from_cube(header):
    data = {}
    data['START_TIME'] = find_in_dict(header, 'StartTime')
    data['SPACECRAFT_NAME'] = find_in_dict(header, 'SpacecraftName')
    data['INSTRUMENT_NAME'] = find_in_dict(header, 'InstrumentId')
    data['SAMPLING_FACTOR'] = find_in_dict(header, 'SpatialSumming')
    data['SAMPLE_FIRST_PIXEL'] = find_in_dict(header, 'SampleFirstPixel')
    data['IMAGE'] = {}
    data['IMAGE']['LINES'] = find_in_dict(header, 'Lines')
    data['IMAGE']['SAMPLES'] = find_in_dict(header, 'Samples')
    data['TARGET_NAME'] = find_in_dict(header, 'TargetName')
    data['LINE_EXPOSURE_DURATION'] = find_in_dict(header, 'LineExposureDuration')
    data['SPACECRAFT_CLOCK_START_COUNT'] = find_in_dict(header, 'SpacecraftClockCount')
    return data

import json
data = data_from_cube(label)
data = json.dumps(data, cls=NumpyEncoder)

In [3]:
print(data)

{"START_TIME": "2017-01-05T07:53:59.568000", "SPACECRAFT_NAME": "Mars_Reconnaissance_Orbiter", "INSTRUMENT_NAME": "CTX", "SAMPLING_FACTOR": 1, "SAMPLE_FIRST_PIXEL": 0, "IMAGE": {"LINES": 6144, "SAMPLES": 3144}, "TARGET_NAME": "Mars", "LINE_EXPOSURE_DURATION": [1.877, "MSEC"], "SPACECRAFT_CLOCK_START_COUNT": "1168070085:007"}


In [6]:
r = requests.post('http://smalls:8002/api/1.0/missions/mars_reconnaissance_orbiter/csm_isd', data=data)
r.json()

{'data': {'isd': '{"DT_EPHEM": 0.15016, "NUMBER_OF_EPHEM": 77, "T0_QUAT": -5.766144037246704, "MIN_VALID_HT": -8000, "SEMI_MAJOR_AXIS": 3396190.0, "QUATERNIONS": [0.738045822819702, -0.1338958889961916, 0.6315594690562354, 0.19619605341014018, 0.7380917445206792, -0.13391105515606164, 0.6315061489802432, 0.19618458088635282, 0.7381363467770333, -0.13392674439856028, 0.6314537675831763, 0.19617466734130037, 0.738180520927701, -0.13394315803618737, 0.6314019935548075, 0.19616388932017007, 0.7382248555117148, -0.1339600852146685, 0.6313499658764672, 0.19615294762447602, 0.7382691222553717, -0.13397639917839804, 0.6312980950254123, 0.1961421494854153, 0.7383140306954811, -0.13399160465028748, 0.6312459437203897, 0.1961305700241246, 0.7383587705366055, -0.13400722809417348, 0.6311944120644651, 0.1961173194003597, 0.7384032271739568, -0.13402296059817756, 0.6311433312036583, 0.19610358387529442, 0.738447187698474, -0.13403795066812302, 0.6310920382664109, 0.196092881047864, 0.738491757572067

## Use the ISD

In [4]:
import usgscam as cam
from cycsm.isd import Isd

In [5]:
isd = r.json()['data']['isd']

In [6]:
i = Isd.loads(isd)

In [7]:
plugin = cam.genericls.GenericLsPlugin()
camera = plugin.from_isd(i, plugin.modelname(0))

In [13]:
camera.imagesize

(18432.0, 5000.0)

In [14]:
xyz = camera.imageToGround(9126, 2500, 0)
xyz

[701107.7192418213, 3142968.175070605, 1072678.7253195439]

In [15]:
camera.groundToImage(*xyz)

[9126.000016312359, 2500.000000206428]

## Random CTX Image

The above could be perceived as being too canned.  Here, pull a random image from the PDS and instantiate a camera model.

In [16]:
from ftplib import FTP
import os
from random import choice

In [41]:
# Run only once to avoid too many open FTP connections
ftp = FTP('pdsimage2.wr.usgs.gov')
ftp.login()
dirs = ftp.nlst('archive/mro-m-ctx-2-edr-l0-v1.0/')

In [42]:
data_dir = choice(dirs)
f = choice(ftp.nlst(data_dir + '/data'))
fname = os.path.basename(f)
with open('/data/ctx_sample/{}'.format(fname), 'wb') as file:
    ftp.retrbinary('RETR {}'.format(f), file.write)

In [43]:
newfile = os.path.join('/data/ctx_sample/{}'.format(fname))
newfile

'/data/ctx_sample/F04_037269_2011_XI_21N056W.IMG'

In [34]:
# Utility Func for working with PVL
def find_in_dict(obj, key):
    """
    Recursively find an entry in a dictionary

    Parameters
    ----------
    obj : dict
          The dictionary to search
    key : str
          The key to find in the dictionary

    Returns
    -------
    item : obj
           The value from the dictionary
    """
    if key in obj:
        return obj[key]
    for k, v in obj.items():
        if isinstance(v,dict):
            item = find_in_dict(v, key)
            if item is not None:
                return item
            
def data_from_cube(header):
    data = {}
    data['START_TIME'] = find_in_dict(header, 'StartTime')
    data['SPACECRAFT_NAME'] = find_in_dict(header, 'SpacecraftName')
    data['INSTRUMENT_NAME'] = find_in_dict(header, 'InstrumentId')
    data['SAMPLING_FACTOR'] = find_in_dict(header, 'SpatialSumming')
    data['SAMPLE_FIRST_PIXEL'] = find_in_dict(header, 'SampleFirstPixel')
    data['IMAGE'] = {}
    data['IMAGE']['LINES'] = find_in_dict(header, 'Lines')
    data['TARGET_NAME'] = find_in_dict(header, 'TargetName')
    data['LINE_EXPOSURE_DURATION'] = find_in_dict(header, 'LineExposureDuration')
    data['SPACECRAFT_CLOCK_START_COUNT'] = find_in_dict(header, 'SpacecraftClockCount')
    return data

In [35]:
import os

# Simple helper function to use the API
def create_data_pacakge(in_file):
    label = pvl.load(in_file)
    return json.dumps(label, cls=NumpyEncoder)

#newfile = '/data/ctx_sample/F04_037269_2011_XI_21N056W.IMG'
newfile = '/data/CTX/calibrated/B01_010045_1878_XN_07N205W.cal.cub'

print('Processing a new image: {}'.format(newfile))
# Call the func to create the data package for submission to the micro-service
data = pvl.load(newfile)
if 'cub' in os.path.splitext(newfile)[1]:
    data = data_from_cube(data)    
data = json.dumps(data, cls=NumpyEncoder)

print(data)
# Call the micro-service
r = requests.post('http://localhost:5000/api/1.0/missions/mars_reconnaissance_oribter/ctx/csm_isd', data=data)

# Get the ISD back and instantiate a local ISD for the image
isd = r.json()['data']['isd']
i = Isd.loads(isd)

# Create the plugin and camera as usual
plugin = cam.genericls.GenericLsPlugin()
camera = plugin.from_isd(i, plugin.modelname(0))

shape = camera.imagesize
# Call I2G
gnd = camera.imageToGround(shape[0] / 2, shape[1] / 2, 0)
print(gnd)

# Call G2I
camera.groundToImage(*gnd)

Processing a new image: /data/CTX/calibrated/B01_010045_1878_XN_07N205W.cal.cub
{"START_TIME": "2008-09-17T05:08:10.820000", "SPACECRAFT_NAME": "Mars_Reconnaissance_Orbiter", "INSTRUMENT_NAME": "CTX", "SAMPLING_FACTOR": 1, "SAMPLE_FIRST_PIXEL": 0, "IMAGE": {"LINES": 7168}, "TARGET_NAME": "Mars", "LINE_EXPOSURE_DURATION": [1.877, "MSEC"], "SPACECRAFT_CLOCK_START_COUNT": "0906095311:038"}
(7168.0, 5000.0)
[-3035183.4483585223, 1452666.3622457734, 457215.05215266405]


[3584.000022561103, 2500.0000033572246]

In [12]:
import pyproj
from math import sqrt

semi_major = i.param("SEMI_MAJOR_AXIS")
semi_minor = semi_major * sqrt(1 - i.param("ECCENTRICITY")**2)

ecef = pyproj.Proj(proj='geocent', a=semi_major, b=semi_minor)
lla = pyproj.Proj(proj='longlat', a=semi_major, b=semi_minor)

gnd = camera.imageToGround(shape[0], shape[1], 0)


lons, lats, alts = pyproj.transform(ecef, lla, gnd[0], gnd[1], gnd[2])
lons, lats

(-56.62606257827925, 21.679360463340597)